# TP Industrias X.0

In [ ]:
# librerias
#!pip install numpy
#!pip install pandas
#!pip install matplotlib.pyplot
#!pip install seaborn
import numpy as np
import pandas as pd
import math as mh
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk

In [ ]:
df_train = pd.read_csv('Bank Marketing train_set.csv')

In [ ]:
df_test = pd.read_csv('Bank Marketing test_set.csv')

## ***Analisis Exploratorio de datos***

In [ ]:
# que columnas tienen los datos?
print(df_train.columns)
cols = df_train.columns

In [ ]:
df_train.head() #observo las primeras 5 filas del df

In [ ]:
# qué tamaño tienen los datos?
print(df_train.shape)

In [ ]:
# hay valores nulos en los datos?
print(df_train.info())
#object tipo string. Int, float son tipo numerico
#valores non null menores a 38429 indican valores nulos en ese campo

In [ ]:
#Valores nulos totales
df_train.isna().sum().sort_values().plot(kind='bar')

Se observan valores **nulos** en las variables: ***age, job, Balance y Pdays***. Principalmente en Age y Balance.

Más adelante, habrá que tratar estos casos de algún modo.

In [ ]:
# como se distribuyen las variables numéricas
print(df_train.describe())

Ademas, podemos hacer una comparación entre media y mediana para analizar si estas distribuciones son simétricas o no (si la media y la mediana son cercanas, la dist es más simétrica).

A mayor simetría de la distribución, más podría aproximarce la variable a una Normal, lo cual es beneficioso en los modelos de Machine Learning más probabilisticos como la Regresión Logística.

In [ ]:
df_train.median().sort_values().plot(kind='barh')

In [ ]:
# diferencia porcentual entre media y mediana para las variables:
((df_train.mean()-df_train.median())/df_train.mean())*100

-------------------------------------------------------
Obsevamos que la mayoría de las variables **no** poseen medias y medianas cercanas, por lo que estarían lejos de ser simétricas.

Se puede continuar este análsis de manera gráfica, armando los histogramas de cada variable:

In [ ]:
# histogramas

for i in cols:
    if (df_train[i].dtype != 'O')&(i!='Last Contact Day'):
        print('Variable:',i)
        plt.figure(figsize=(10,5))
        sns.histplot(df_train[i])
        plt.show()


En los histogramas de varias variables se observa mucha asimetría positiva (similar a dist log-Normal), por lo que si más adelante necesitáramos que estas distribuaciones se asemejen a una Normal, podríamos aplicar una transformación (ej logarítmica) para volverlas más asimétricas.

In [ ]:
#sns.pairplot(df_train[['Age','Balance (euros)','Housing Loan','Last Contact Day','Last Contact Duration','Campaign','Pdays','Previous','Subscription']], hue = 'Subscription')

In [ ]:
#estudio de correlacion de variables con valores nulos vs Y (suscrpcion)
#df_train_nan = df_train[['Age','Job','Balance (euros)','Pdays','Subscription']] #variables con NAs en train
sns.heatmap(df_train[['Age','Balance (euros)','Housing Loan','Last Contact Day','Last Contact Duration','Campaign','Pdays','Previous','Subscription']].corr(), annot=True, cmap="YlGnBu")
plt.show()

No Hay correlaciones fuerte 1o1 entre las variables, siendo significativamente superior la correlacion entre suscripcion y last call duration, pdays y previous y en menor medida campaign con last contact day.

In [ ]:
# nro de valores únicos por variable
df_train.nunique().sort_values()

In [ ]:
# como se comportan las variables categóricas
df_train.describe(include=['O'])

Vemos que el trabajo mas frecuente es blue-collar, hay mayor cantidad de casados que solteros, la mayoria tuvo al menos educacion secundaria y pidieron prestamos hipotecarios. La myoria no defaulteo ni tiene prestamos personales. El contacto de preferencia es por celular. El mes de mayor cantidad de contactos fue Mayo.

In [ ]:
# grafico de barras con las etiquetas de las cantidades
fig_ms, ax_ms = plt.subplots()
y_ms = df_train['Marital Status'].value_counts()

hbars_ms = ax_ms.barh(y_ms.keys(), y_ms, align='center')
ax_ms.invert_yaxis()  # para que aparezca primero las cat de mayor frec
ax_ms.set_xlim(right=max(y_ms)+0.25*max(y_ms))
ax_ms.set_title('¿Cuánta gente está casada, soltera y divorciada?')

#ax_ms.bar_label(hbars_ms) # así podría agregar labels si tuviera la ultima version de matplotlib
for i, v in enumerate(y_ms):
    plt.text(v, i, "  " + str(v), color='black', va='center', fontweight='bold')

plt.show()

In [ ]:
# grafico de barras con las etiquetas de las cantidades
fig_ed, ax_ed = plt.subplots()
y_ed = df_train['Education'].value_counts()

hbars_ed = ax_ed.barh(y_ed.keys(), y_ed, align='center')
ax_ed.invert_yaxis()  # para que aparezca primero las cat de mayor frec
ax_ed.set_xlim(right=max(y_ed)+0.25*max(y_ed))
ax_ed.set_title('¿Cuánta gente tiene educación primaria, secundaria y terciaria?')

#ax_ed.bar_label(hbars_ed) # así podría agregar labels si tuviera la ultima version de matplotlib
for i, v in enumerate(y_ed):
    plt.text(v, i, "  " + str(v), color='black', va='center', fontweight='bold')

plt.show()

In [ ]:
#cambio valores 1 y 2 a binario 0, 1 para decir si estan suscritos o no
df_train['Subscription']=df_train['Subscription'].replace(1,0)
df_train['Subscription']=df_train['Subscription'].replace(2,1)
df_train.groupby(['Subscription']).count()['Education']

In [ ]:
# grafico de barras con las etiquetas de las cantidades
fig_sub, ax_sub = plt.subplots()
y_sub = df_train['Subscription'].value_counts()

hbars_sub = ax_sub.barh(y_sub.keys().astype(str), y_sub, align='center')
ax_sub.invert_yaxis()  # para que aparezca primero las cat de mayor frec
ax_sub.set_xlim(right=max(y_sub)+0.25*max(y_sub))
ax_sub.set_title('¿Cuánta gente se suscribió al plazo fijo y cuantas no?')

#ax_sub.bar_label(hbars_sub) # así podría agregar labels si tuviera la ultima version de matplotlib
for i, v in enumerate(y_sub):
    plt.text(v, i, "  " + str(v), color='black', va='center', fontweight='bold')

plt.show()

## ***Limpieza de los datos***


In [ ]:
df_train.isna().sum().sort_values() #train

In [ ]:
df_test.isna().sum() #test

In [ ]:
df_train.groupby(['Subscription','Job']).count()['Education']

Unknown representa un % muy bajo para ambos grupos, suscritos y no suscritos

Vamos a generar 3 dataframes, 
1) eliminando datos faltantes ya que hay baja correlacion con la variable a predecir.

2) reemplazando los valores faltantes por mediana en caso de series numericas y "Desconocido" para categoricas.

3) reemplazando los valores faltantes numericos por el promedio y "Desconocido" para categoricas.

In [ ]:
#caso numero 1 
df_train_1 = df_train.copy()
df_train_1 = df_train_1.dropna()

In [ ]:
df_train_1.isna().sum()

In [ ]:
#caso numero 2 
l1 =[]
for i in ['Age','Balance (euros)','Pdays']:
    m = df_train[i].median()
    l1.append(m)
print(l1)

In [ ]:
df_train_2 = df_train.copy()
df_train_2['Age']=df_train_2['Age'].fillna(l1[0])
df_train_2['Balance (euros)']=df_train_2['Balance (euros)'].fillna(l1[1])
df_train_2['Pdays']=df_train_2['Pdays'].fillna(l1[2])
df_train_2['Job']=df_train_2['Job'].fillna('unknown')

In [ ]:
df_train_2.isna().sum()

In [ ]:
#caso numero 3 

l2 =[]
for i in ['Age','Balance (euros)','Pdays']:
    m = df_train[i].mean()
    l2.append(round(m,2))
print(l2)

In [ ]:
df_train_3 = df_train.copy()
df_train_3['Age']=df_train_2['Age'].fillna(l2[0])
df_train_3['Balance (euros)']=df_train_2['Balance (euros)'].fillna(l2[1])
df_train_3['Pdays']=df_train_2['Pdays'].fillna(l2[2])
df_train_3['Job']=df_train_2['Job'].fillna('unknown')

In [ ]:
df_train_3.isna().sum()

In [ ]:
#nuevas variables 

# podemos agrupar las dos variables de prestamos para saber si tiene 1 2 o ninguno 
    # 1+1 = 2
    # 1+0 = 1
    # 0+1 = 1
    # 0+0 = 0
# tenemos deudor nivel 0, 1 y 2
#df_train_p=df_train.copy()

# set de training
df_train_1['Loan']=np.where(df_train_1['Loan']=='yes',1,0)
df_train_1['Housing Loan']=np.where(df_train_1['Housing Loan']=='yes',1,0)
df_train_1['Total_loan']=df_train_1['Loan']+df_train_1['Housing Loan']

df_train_2['Loan']=np.where(df_train_2['Loan']=='yes',1,0)
df_train_2['Housing Loan']=np.where(df_train_2['Housing Loan']=='yes',1,0)
df_train_2['Total_loan']=df_train_2['Loan']+df_train_2['Housing Loan']

df_train_3['Loan']=np.where(df_train_3['Loan']=='yes',1,0)
df_train_3['Housing Loan']=np.where(df_train_3['Housing Loan']=='yes',1,0)
df_train_3['Total_loan']=df_train_3['Loan']+df_train_3['Housing Loan']

# set de testing
df_test['Loan']=np.where(df_test['Loan']=='yes',1,0)
df_test['Housing Loan']=np.where(df_test['Housing Loan']=='yes',1,0)
df_test['Total_loan']=df_test['Loan']+df_test['Housing Loan']


In [ ]:
def set_values(row, value):
    return value[row]
   
map_dictionary ={0 : 'nivel 0', 1 :'nivel 1', 2 : "nivel 2"} 
   
df_train_1['Deudor'] = df_train_1['Total_loan'].apply(set_values, args =(map_dictionary, ))
df_train_2['Deudor'] = df_train_2['Total_loan'].apply(set_values, args =(map_dictionary, ))
df_train_3['Deudor'] = df_train_3['Total_loan'].apply(set_values, args =(map_dictionary, ))
df_test['Deudor'] = df_test['Total_loan'].apply(set_values, args =(map_dictionary, ))
df_train_1.drop('Total_loan',axis=1,inplace=True)
df_train_2.drop('Total_loan',axis=1,inplace=True)
df_train_3.drop('Total_loan',axis=1,inplace=True)


NameError: ignored

In [ ]:
df_train_1['Deudor'].head()

In [ ]:
df_train_2['Deudor'].head()

In [ ]:
df_train_3['Deudor'].head()

In [ ]:
df_test['Deudor'].head()

In [ ]:
df_train_1.groupby(['Subscription','Deudor']).count()['Education'].sort_index()

In [ ]:
df_train_1['Deudor'].value_counts().sort_index()

In [ ]:
df_train_2['Deudor'].value_counts().sort_index()

In [ ]:
df_train_3['Deudor'].value_counts().sort_index()

In [ ]:
# Exportar a CSV
df_train_1.to_csv('datos_nulos_eliminados.csv',index=False)
df_train_2.to_csv('datos_nulos_reemplazados_mediana.csv',index=False)
df_train_3.to_csv('datos_nulos_reemplazados_promedio.csv',index=False)

Dado que observamos mucha asimetría en las variables, lo más representativo sería utilizar la mediana para reemplazar los NA.

## ***Modelos de Machine Learning***

#### Se realizan 3 modelos de aprendizaje supervisado para cada uno de los datasets generados previamente

In [ ]:
# uso el set de datos en el que se reemplazo los nulos por la mediana
lista_archivos = [df_train_1,df_train_2,df_train_3]


### Preprocesamiento para Modelos

In [ ]:
#dummies priemr dataframe
x_train_ml_1 = df_train_1.loc[:, df_train_1.columns != 'Subscription'] # variables explicativas
y_train_ml_1 = df_train_1[['Subscription']] # variables de respuesta
x_train_ml_1.head()
print(x_train_ml_1[['Job']].value_counts(), '\n \n',
x_train_ml_1[['Marital Status']].value_counts(), '\n \n',
x_train_ml_1[['Education']].value_counts(), '\n \n',
x_train_ml_1[['Default']].value_counts(), '\n \n',
x_train_ml_1[['Contact']].value_counts(), '\n \n',
x_train_ml_1[['Poutcome']].value_counts(), '\n \n',
x_train_ml_1[['Deudor']].value_counts())
# Codifico dummies tomando referencia en la categoría más común
x_train_ml_dum_1 = pd.get_dummies(x_train_ml_1).drop(['Job_blue-collar',
                                              'Marital Status_married',
                                              'Education_secondary',
                                              'Default_no',
                                              'Contact_cellular',
                                              'Poutcome_unknown',
                                              'Deudor_nivel 1'],
                                             axis=1)
# hago lo mismo para el set de testing
df_test_dum_1 = pd.get_dummies(df_test).drop(['Job_blue-collar',
                                            'Marital Status_married',
                                            'Education_secondary',
                                            'Default_no',
                                            'Contact_cellular',
                                            'Poutcome_unknown',
                                            'Deudor_nivel 1'],
                                             axis=1)
# analizo las categorias y las cantidades de datos para cada categoría de las variables categóricas
# para que me ayude a definir categorias de referencia/base para codificar dummies
print(x_train_ml_1[['Job']].value_counts(), '\n \n',
x_train_ml_1[['Marital Status']].value_counts(), '\n \n',
x_train_ml_1[['Education']].value_counts(), '\n \n',
x_train_ml_1[['Default']].value_counts(), '\n \n',
x_train_ml_1[['Contact']].value_counts(), '\n \n',
x_train_ml_1[['Poutcome']].value_counts(), '\n \n',
x_train_ml_1[['Deudor']].value_counts())
# Codifico dummies tomando referencia en la categoría más común
x_train_ml_dum_1 = pd.get_dummies(x_train_ml_1).drop(['Job_blue-collar',
                                                  'Marital Status_married',
                                                  'Education_secondary',
                                                  'Default_no',
                                                  'Contact_cellular',
                                                  'Poutcome_unknown',
                                                  'Deudor_nivel 1'],
                                                   axis=1)
# hago lo mismo para el set de testing
df_test_dum_1 = pd.get_dummies(df_test).drop(['Job_blue-collar',
                                                'Marital Status_married',
                                                'Education_secondary',
                                                'Default_no',
                                                'Contact_cellular',
                                                'Poutcome_unknown',
                                                'Deudor_nivel 1'],
                                                 axis=1)
#print(x_train_ml_dum_1.head(5)) 
#print(y_train_ml_1.head(5))

In [ ]:
#dummies segundo dataframe
x_train_ml_2 = df_train_2.loc[:, df_train_2.columns != 'Subscription'] # variables explicativas
y_train_ml_2 = df_train_2[['Subscription']] # variables de respuesta
'''x_train_ml_2.head()
print(x_train_ml_2[['Job']].value_counts(), '\n \n',
x_train_ml_2[['Marital Status']].value_counts(), '\n \n',
x_train_ml_2[['Education']].value_counts(), '\n \n',
x_train_ml_2[['Default']].value_counts(), '\n \n',
x_train_ml_2[['Contact']].value_counts(), '\n \n',
x_train_ml_2[['Poutcome']].value_counts(), '\n \n',
x_train_ml_2[['Deudor']].value_counts())'''
# Codifico dummies tomando referencia en la categoría más común
x_train_ml_dum_2 = pd.get_dummies(x_train_ml_2).drop(['Job_blue-collar',
                                              'Marital Status_married',
                                              'Education_secondary',
                                              'Default_no',
                                              'Contact_cellular',
                                              'Poutcome_unknown',
                                              'Deudor_nivel 1'],
                                             axis=1)
# hago lo mismo para el set de testing
df_test_dum_2 = pd.get_dummies(df_test).drop(['Job_blue-collar',
                                            'Marital Status_married',
                                            'Education_secondary',
                                            'Default_no',
                                            'Contact_cellular',
                                            'Poutcome_unknown',
                                            'Deudor_nivel 1'],
                                             axis=1)
# analizo las categorias y las cantidades de datos para cada categoría de las variables categóricas
# para que me ayude a definir categorias de referencia/base para codificar dummies
'''print(x_train_ml_2[['Job']].value_counts(), '\n \n',
x_train_ml_2[['Marital Status']].value_counts(), '\n \n',
x_train_ml_2[['Education']].value_counts(), '\n \n',
x_train_ml_2[['Default']].value_counts(), '\n \n',
x_train_ml_2[['Contact']].value_counts(), '\n \n',
x_train_ml_2[['Poutcome']].value_counts(), '\n \n',
x_train_ml_2[['Deudor']].value_counts())'''
# Codifico dummies tomando referencia en la categoría más común
x_train_ml_dum_2 = pd.get_dummies(x_train_ml_2).drop(['Job_blue-collar',
                                                  'Marital Status_married',
                                                  'Education_secondary',
                                                  'Default_no',
                                                  'Contact_cellular',
                                                  'Poutcome_unknown',
                                                  'Deudor_nivel 1'],
                                                   axis=1)
# hago lo mismo para el set de testing
df_test_dum_2 = pd.get_dummies(df_test).drop(['Job_blue-collar',
                                                'Marital Status_married',
                                                'Education_secondary',
                                                'Default_no',
                                                'Contact_cellular',
                                                'Poutcome_unknown',
                                                'Deudor_nivel 1'],
                                                 axis=1)
#x_train_ml_dum_2.head(10) 

In [ ]:
#dummies segundo dataframe
x_train_ml_3 = df_train_3.loc[:, df_train_3.columns != 'Subscription'] # variables explicativas
y_train_ml_3 = df_train_3[['Subscription']] # variables de respuesta
'''x_train_ml_3.head()
print(x_train_ml_3[['Job']].value_counts(), '\n \n',
x_train_ml_3[['Marital Status']].value_counts(), '\n \n',
x_train_ml_3[['Education']].value_counts(), '\n \n',
x_train_ml_3[['Default']].value_counts(), '\n \n',
x_train_ml_3[['Contact']].value_counts(), '\n \n',
x_train_ml_3[['Poutcome']].value_counts(), '\n \n',
x_train_ml_3[['Deudor']].value_counts())'''
# Codifico dummies tomando referencia en la categoría más común
x_train_ml_dum_3 = pd.get_dummies(x_train_ml_3).drop(['Job_blue-collar',
                                              'Marital Status_married',
                                              'Education_secondary',
                                              'Default_no',
                                              'Contact_cellular',
                                              'Poutcome_unknown',
                                              'Deudor_nivel 1'],
                                             axis=1)
# hago lo mismo para el set de testing
df_test_dum_3 = pd.get_dummies(df_test).drop(['Job_blue-collar',
                                            'Marital Status_married',
                                            'Education_secondary',
                                            'Default_no',
                                            'Contact_cellular',
                                            'Poutcome_unknown',
                                            'Deudor_nivel 1'],
                                             axis=1)
# analizo las categorias y las cantidades de datos para cada categoría de las variables categóricas
# para que me ayude a definir categorias de referencia/base para codificar dummies
'''print(x_train_ml_3[['Job']].value_counts(), '\n \n',
x_train_ml_3[['Marital Status']].value_counts(), '\n \n',
x_train_ml_3[['Education']].value_counts(), '\n \n',
x_train_ml_3[['Default']].value_counts(), '\n \n',
x_train_ml_3[['Contact']].value_counts(), '\n \n',
x_train_ml_3[['Poutcome']].value_counts(), '\n \n',
x_train_ml_3[['Deudor']].value_counts())'''
# Codifico dummies tomando referencia en la categoría más común
x_train_ml_dum_3 = pd.get_dummies(x_train_ml_3).drop(['Job_blue-collar',
                                                  'Marital Status_married',
                                                  'Education_secondary',
                                                  'Default_no',
                                                  'Contact_cellular',
                                                  'Poutcome_unknown',
                                                  'Deudor_nivel 1'],
                                                   axis=1)
# hago lo mismo para el set de testing
df_test_dum_3 = pd.get_dummies(df_test).drop(['Job_blue-collar',
                                                'Marital Status_married',
                                                'Education_secondary',
                                                'Default_no',
                                                'Contact_cellular',
                                                'Poutcome_unknown',
                                                'Deudor_nivel 1'],
                                                 axis=1)
#x_train_ml_dum_3.head(10) 

In [ ]:
x_dummie=[x_train_ml_dum_1,x_train_ml_dum_2,x_train_ml_dum_3]
y_dummie=[y_train_ml_1,y_train_ml_2,y_train_ml_3]


A falta de conocimiento sobre un mejor criterio, vamos a tomar la categoría con más datos de cada variable como **referencia para la codificación de dummies**, ya que se supone como el caso más común para tomar de base.

In [ ]:
# modulos necesarios para preprocesamiento
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

### Modelo 1: ***Regresion logistica***

 Dado la **gran diferencia en ordenes de magnitud de las distintas variables numéricas/continuas** de los datasets. Lo mejor será **estandarizar/escalar** estas variables para entrenar y predecir con los modelos de machine learning.

In [ ]:
variables_num = x_train_ml_dum_2[['Age', 'Balance (euros)', 'Last Contact Day', 'Last Contact Duration', 'Campaign', 'Pdays']].columns.values
print(variables_num)

In [ ]:
# estandarizo/escalo las variables continuas
scaler = StandardScaler()
x_train_ml_Num = x_train_ml_dum_2[variables_num]
df_test_Num = df_test_dum_2[variables_num]

x_train_ml_NumScaled = scaler.fit_transform(x_train_ml_Num)
df_test_NumScaled = scaler.transform(df_test_Num)

# corroboro que estan estandarizados (aprox media 0 y desvio 1):
print("Train: media "+ str(x_train_ml_NumScaled.mean())+ " y desvio "+ str(x_train_ml_NumScaled.std()) +"\n"+
     "Test: media " + str(df_test_NumScaled.mean()) +" y desvio "+ str(df_test_NumScaled.std()) )

In [ ]:
# convierto en dataframe porque es más facil para trabajar 
x_train_ml_NumScaled = pd.DataFrame(x_train_ml_NumScaled, columns = list(x_train_ml_dum_2[variables_num].columns))
df_test_NumScaled = pd.DataFrame(df_test_NumScaled, columns = list(df_test_dum_2[variables_num].columns))

In [ ]:
x_train_ml_NumScaled.head()

In [ ]:
df_test_NumScaled.head()

In [ ]:
# junto el df de variables numéricas estandarizadas/escaladas con el df de dummies
x_train_ml_dum_scaled = pd.concat([x_train_ml_NumScaled, x_train_ml_dum_2.drop(variables_num, axis=1)], axis=1)
df_test_dum_scaled = pd.concat([df_test_NumScaled, df_test_dum_2.drop(variables_num, axis=1)], axis=1)

In [ ]:
# data set de training con dummies codificadas y variables numéricas estandarizadas
x_train_ml_dum_scaled.head(10)

In [ ]:
# data set de testing con dummies codificadas y variables numéricas estandarizadas
del df_test_dum_scaled['Total_loan'] # no lo necesito por la correlacion con Deudor
df_test_dum_scaled.head(10)

In [ ]:
# otros modulos necesarios para preprocesamiento
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold #útil para casos como este donde la variable de respuesta está desbalanceada
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# NO uso las transformaciones anteriores porque me está tirando NAs, no entiendo bien por que

In [ ]:
# valores de los hiperparámetros posibles para la regresión logística
# (tarda unos 10/15 minutos en correr todos los modelos con los distintos parametros)
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear'] # algoritmo de convergencia
penalty = ['l2'] # tipo de penalidad (regularización)
c_values = [10000000, 1000000, 100000, 10000, 1000] # valores del hiperparámetro para penalidad

grid = dict(solver=solvers,penalty=penalty,C=c_values) # armo la grid search

# armo los folds de la cross validation de manera estratificada porque son datos desbalanceados
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1) 

# defino la grid search con los parametros a probar y la cross validation
grid_search_logreg = GridSearchCV(estimator=model,
                           param_grid=grid,
                           n_jobs=-1, # (solo tengo un procesador en mi compu pero aca le pondría que lo corra en todos)
                           cv=cv,
                           scoring='f1', # como esta desbalanceado uso f1 = 2* (precision*recall)/(precision+recall)
                           error_score=0)

In [ ]:
# entreno el modelo
grid_result_logreg = grid_search_logreg.fit(x_train_ml_dum_scaled.values, y_train_ml_2.values.ravel())

In [ ]:
# mejor resultado
print("Mejor: %f usando %s" % (grid_result_logreg.best_score_, grid_result_logreg.best_params_))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
x_train_train_ml_dum_scaled, x_test_train_ml_dum_scaled, y_train_train_ml_2, y_test_train_ml_2 = train_test_split(x_train_ml_dum_scaled, 
                                                                                                                  y_train_ml_2,
                                                                                                                  test_size=0.25,
                                                                                                                  random_state=0)


# armo la regresión logística con los mejores hiperparámetros encontrados
logreg = LogisticRegression(C= 10000000, penalty = 'l2', solver='newton-cg')

logreg = logreg.fit(x_train_train_ml_dum_scaled.values, y_train_train_ml_2.values.ravel())

In [ ]:
# Matriz de Confusión y cálculo de scores para logreg
y_pred_logreg = logreg.predict(x_test_train_ml_dum_scaled)
#matriz de confusion
#from slkearn.metrics import confusion_matrix 
print('Matriz de confusion')
print('-------------------')
cm_logreg = confusion_matrix(y_test_train_ml_2,y_pred_logreg)
print(cm_logreg)



In [ ]:
y_pred_logreg_final = logreg.predict(df_test_dum_scaled)

### Modelo 2: ***Bayes Ingenuo***

In [ ]:
#librerias  
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_confusion_matrix


In [ ]:
cont=0
for i in x_dummie:
    print('================================================')
    if cont==0:
        print('Analisis numero 1')
        print('Modelo con dataset eliminando datos faltantes ya que hay baja correlacion con la variable a predecir y tienen poco peso sobre el total de los datos')
        print('================================================')
    elif cont==1: 
        print('Analisis numero 2')
        print('Modelo con dataset reemplazando los valores faltantes por mediana en caso de series numericas y "Desconocido" para categoricas.')
        print('================================================')
    else:
        print('Analisis numero 3')
        print('Modelo con dataset reemplazando los valores faltantes numericos por el promedio y "Desconocido" para categoricas.')
        print('================================================')
   
    #divido en prueba y testeo
    #from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(i, 
                                          y_dummie[cont],
                                          test_size=0.25,
                                          random_state=0)
    #estandarizacion de escalas
    #from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.fit_transform(x_test)
    #defino el objeto clasificador
    from sklearn.naive_bayes import GaussianNB
    classifier = GaussianNB()   
    #entreno el clasificador
    classifier.fit(x_train,y_train)   
    #predigo usando las x que guarde como test 
    y_pred_bayes = classifier.predict(x_test)
    #matriz de confusion
    #from slkearn.metrics import confusion_matrix 
    print('Matriz de confusion')
    print('-------------------')
    cm_bayes = confusion_matrix(y_test,y_pred_bayes)
    print(cm_bayes)
    #plot_confusion_matrix(classifier, x_train, y_train, cmap=plt.cm.Blues, values_format = '.0f')
    
    cont+=1

### Modelo 3: ***Árbol de Decisión***

In [ ]:
cont=0
arboles=[]
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
for i in x_dummie:
    print('================================================')
    if cont==0:
        print('Analisis numero 1')
        print('Modelo con dataset eliminando datos faltantes ya que hay baja correlacion con la variable a predecir y tienen poco peso sobre el total de los datos')
        print('================================================')
    elif cont==1:
        print('Analisis numero 2')
        print('Modelo con dataset reemplazando los valores faltantes por mediana en caso de series numericas y "Desconocido" para categoricas.')
        print('================================================')
    else:
        print('Analisis numero 3')
        print('Modelo con dataset reemplazando los valores faltantes numericos por el promedio y "Desconocido" para categoricas.')
        print('================================================')
    #divido en prueba y testeo
    #from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(i, 
                                          y_dummie[cont],
                                          test_size=0.25,
                                          random_state=0)
    #estandarizacion
    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.fit_transform(x_test)
    #from sklearn.tree import DecisionTreeClassifier
    clf = DecisionTreeClassifier(random_state=0)
    clf = clf.fit(x_train,y_train)
    '''print('Relevancia de cada campo:')
    feature_names = x_train.columns
    featureImportance = pd.DataFrame(clf.feature_importances_, index=feature_names)
    featureImportance.head(20)
    featureImportance.plot(kind='bar')'''
    print('---------------------------')
    print('Parametros del modelo')
    print('---------------------------')
    print(clf.get_params())
    y_pred_ad = clf.predict(x_test) 
    #performance 
    print('---------------------------')
    print(' Evaluacion de performance')
    print('---------------------------')
    #from sklearn.metrics import accuracy_score      
    acSc = accuracy_score(y_test, y_pred_ad)
    print('Accuracy = %s'%(acSc))  
    print('---------------------------')
    print('Matriz de confusion')
    #matriz de confusion
    #from slkearn.metrics import confusion_matrix 
    cm_ad = confusion_matrix(y_test,y_pred_ad)
    print(cm_ad)
    print('---------------------------')
    #precision
    #from sklearn.metrics import precision_score
    prec = precision_score(y_test,y_pred_ad)
    print('Precision: %s'%(prec))
    print('---------------------------')
    #recall score
    #from sklearn.metrics import recall_score
    rec = recall_score(y_test,y_pred_ad)
    print('Recall Score: %s'%(rec))
    #reporte de performance total
    #from sklearn.metrics import classification_report
    #print('---------------------------')
    #print('Reporte de resultados:')
    #print('---------------------------')
    #classification_report(y_test, y_pred_ad)
    #grafico del arbol
    #'''print('Grafico')
    #print('---------------------------')
    #from sklearn import tree
    #fig = plt.figure(figsize=(25,20))
    #treePlot = clf.plot_tree(clf,
    #                          feature_names=feature_names,
    #                          class_names={0:'No suscripcion',1:'Suscripto'},
    #                          filled = True,
    #                          fontsize = 10)
    #treePlot.show()   ''' 
    #optimizar modelo
    print('---------------------------')
    print('Optimizador de parametros')
    print('---------------------------')
    if cont==0:
        #from sklearn.model_selection import cross_val_score
        tree1 = DecisionTreeClassifier()
        tree_scores = cross_val_score(tree1, i, y_dummie[cont], cv=5)
        #Max depth optimo
        #from sklearn.model_selection import cross_validate
        tree_train_scores_mean = []
        tree_train_scores_std = []
        tree_test_scores_mean = []
        tree_test_scores_std = []

        profundidades = np.arange(1,50,1)

        for profundidad in profundidades:
            clf = DecisionTreeClassifier(max_depth=profundidad)
            tree_scores = cross_validate(clf, i, y_dummie[cont], cv=5, return_train_score=True, n_jobs = -1)

            tree_train_scores_mean.append(tree_scores['train_score'].mean())
            tree_train_scores_std.append(tree_scores['train_score'].std())

            tree_test_scores_mean.append(tree_scores['test_score'].mean())
            tree_test_scores_std.append(tree_scores['test_score'].std())

        tree_train_scores_mean = np.array(tree_train_scores_mean)
        tree_train_scores_std = np.array(tree_train_scores_std)
        tree_test_scores_mean = np.array(tree_test_scores_mean)
        tree_test_scores_std = np.array(tree_test_scores_std)

        plt.figure(figsize=(6,4))
        plt.fill_between(profundidades, tree_train_scores_mean - tree_train_scores_std,
                         tree_train_scores_mean + tree_train_scores_std, alpha=0.1,
                         color="r")
        plt.fill_between(profundidades, tree_test_scores_mean - tree_test_scores_std,
                         tree_test_scores_mean + tree_test_scores_std, alpha=0.1, color="g")
        plt.plot(profundidades, tree_train_scores_mean, 'o-', color="r",
                 label="Training score")
        plt.plot(profundidades, tree_test_scores_mean, 'o-', color="g",
                 label="Test score")
        plt.legend()
        plt.ylabel('Accuracy')
        plt.xlabel('Profundidad Arbol de Decision')
        plt.show()
        arboles.append(tree1)
        cont+=1
    elif cont==1:
        #from sklearn.model_selection import cross_val_score
        tree2 = DecisionTreeClassifier()
        tree_scores = cross_val_score(tree2, i, y_dummie[cont], cv=5)
        #Max depth optimo
        #from sklearn.model_selection import cross_validate
        tree_train_scores_mean = []
        tree_train_scores_std = []
        tree_test_scores_mean = []
        tree_test_scores_std = []

        profundidades = np.arange(1,50,1)

        for profundidad in profundidades:
            clf = DecisionTreeClassifier(max_depth=profundidad)
            tree_scores = cross_validate(clf, i, y_dummie[cont], cv=5, return_train_score=True, n_jobs = -1)

            tree_train_scores_mean.append(tree_scores['train_score'].mean())
            tree_train_scores_std.append(tree_scores['train_score'].std())

            tree_test_scores_mean.append(tree_scores['test_score'].mean())
            tree_test_scores_std.append(tree_scores['test_score'].std())

        tree_train_scores_mean = np.array(tree_train_scores_mean)
        tree_train_scores_std = np.array(tree_train_scores_std)
        tree_test_scores_mean = np.array(tree_test_scores_mean)
        tree_test_scores_std = np.array(tree_test_scores_std)

        plt.figure(figsize=(6,4))
        plt.fill_between(profundidades, tree_train_scores_mean - tree_train_scores_std,
                         tree_train_scores_mean + tree_train_scores_std, alpha=0.1,
                         color="r")
        plt.fill_between(profundidades, tree_test_scores_mean - tree_test_scores_std,
                         tree_test_scores_mean + tree_test_scores_std, alpha=0.1, color="g")
        plt.plot(profundidades, tree_train_scores_mean, 'o-', color="r",
                 label="Training score")
        plt.plot(profundidades, tree_test_scores_mean, 'o-', color="g",
                 label="Test score")
        plt.legend()
        plt.ylabel('Accuracy')
        plt.xlabel('Profundidad Arbol de Decision')
        plt.show()
        arboles.append(tree2)
        cont+=1
    else:
        #from sklearn.model_selection import cross_val_score
        tree3 = DecisionTreeClassifier()
        tree_scores = cross_val_score(tree3, i, y_dummie[cont], cv=5)
        #Max depth optimo
        #from sklearn.model_selection import cross_validate
        tree_train_scores_mean = []
        tree_train_scores_std = []
        tree_test_scores_mean = []
        tree_test_scores_std = []

        profundidades = np.arange(1,50,1)

        for profundidad in profundidades:
            clf = DecisionTreeClassifier(max_depth=profundidad)
            tree_scores = cross_validate(clf, i, y_dummie[cont], cv=5, return_train_score=True, n_jobs = -1)

            tree_train_scores_mean.append(tree_scores['train_score'].mean())
            tree_train_scores_std.append(tree_scores['train_score'].std())

            tree_test_scores_mean.append(tree_scores['test_score'].mean())
            tree_test_scores_std.append(tree_scores['test_score'].std())

        tree_train_scores_mean = np.array(tree_train_scores_mean)
        tree_train_scores_std = np.array(tree_train_scores_std)
        tree_test_scores_mean = np.array(tree_test_scores_mean)
        tree_test_scores_std = np.array(tree_test_scores_std)

        plt.figure(figsize=(6,4))
        plt.fill_between(profundidades, tree_train_scores_mean - tree_train_scores_std,
                         tree_train_scores_mean + tree_train_scores_std, alpha=0.1,
                         color="r")
        plt.fill_between(profundidades, tree_test_scores_mean - tree_test_scores_std,
                         tree_test_scores_mean + tree_test_scores_std, alpha=0.1, color="g")
        plt.plot(profundidades, tree_train_scores_mean, 'o-', color="r",
                 label="Training score")
        plt.plot(profundidades, tree_test_scores_mean, 'o-', color="g",
                 label="Test score")
        plt.legend()
        plt.ylabel('Accuracy')
        plt.xlabel('Profundidad Arbol de Decision')
        plt.show()
        arboles.append(tree3)
        cont+=1

Vemos que la maxima profunidad para la cual no se ve un incremento de accuracy es aproximadamente en 25. A conitnuacion se busca optimizar los parametros del modelo; para mejorar el tiempo de procesamiento dejamos fijo el max depth en 25.

In [ ]:
for i in [tree1,tree2,tree3]:
    print(i.get_params())

In [ ]:
# ESTRATEGIA : Grid Search
#dataset 1

param_grid = {'max_depth':[25],
              'criterion': ['gini', 'entropy'], 
              'min_samples_split':[4,5,6,7,8],
              'min_samples_leaf':[4,5,6,7,8]}
model1 = GridSearchCV(arboles[0], param_grid=param_grid, cv=5)
# Entrenamos: Arbol de decision con la grilla definida arriba y CV con tamaño de Fold=5
model1.fit(x_train, y_train)
print('---------------------------')
print("Mejores parametros: "+str(model1.best_params_))
print('---------------------------')
print("Mejor Score: "+str(round(model1.best_score_,4))+'\n')
scores1 = pd.DataFrame(model1.cv_results_)
scores1
prediction1 = model1.predict(x_test)
print('---------------------------')
print("Matriz de confusión:")
print('---------------------------')
cm1_ad= confusion_matrix(y_test,prediction1)
print(cm1_ad)
print('---------------------------')
print('Accuracy Score:', accuracy_score(y_test, prediction1))
print('---------------------------')
print('Precision Score:', precision_score(y_test, prediction1))
print('---------------------------')
print('Recall Score:', recall_score(y_test, prediction1))
print('---------------------------')
# Reporte de Clasificacion
#print("Reporte de Clasificación:")
#print('---------------------------')
#print(classification_report(y_test, prediction1))

In [ ]:
# ESTRATEGIA : Grid Search
#dataset 2

param_grid2 = {'max_depth':[25],
              'criterion': ['gini', 'entropy'], 
              'min_samples_split':[4,5,6,7,8],
              'min_samples_leaf':[4,5,6,7,8,]}
model2 = GridSearchCV(arboles[1], param_grid=param_grid2, cv=5)
# Entrenamos: Arbol de decision con la grilla definida arriba y CV con tamaño de Fold=5
model2.fit(x_train, y_train)
print('---------------------------')
print("Mejores parametros: "+str(model2.best_params_))
print("Best Score: "+str(round(model2.best_score_,4))+'\n')
scores2 = pd.DataFrame(model2.cv_results_)
scores2
prediction2 = model2.predict(x_test)
print('---------------------------')
print("Matriz de confusión:")
print('---------------------------')
cm2_ad= confusion_matrix(y_test,prediction2)
print(cm2_ad)
print('---------------------------')
print('Accuracy Score:', accuracy_score(y_test, prediction2))
print('---------------------------')
print('Precision Score:', precision_score(y_test, prediction2))
print('---------------------------')
print('Recall Score:', recall_score(y_test, prediction2))
print('---------------------------')
# Reporte de Clasificacion
#print("Reporte de Clasificación:")
#print('---------------------------')
#print(classification_report(y_test, prediction2))

In [ ]:
# ESTRATEGIA : Grid Search
#dataset 3

param_grid = {'max_depth':[25],
              'criterion': ['gini', 'entropy'], 
              'min_samples_split':[4,5,6,7,8],
              'min_samples_leaf':[4,5,6,7,8,]}
model3 = GridSearchCV(arboles[2], param_grid=param_grid, cv=5)
# Entrenamos: Arbol de decision con la grilla definida arriba y CV con tamaño de Fold=5
model3.fit(x_train, y_train)
print('---------------------------')
print("Mejores parametros: "+str(model3.best_params_))
print('---------------------------')
print("Mejor Score: "+str(model3.best_score_)+'\n')
print('---------------------------')
scores3 = pd.DataFrame(model3.cv_results_)
scores3
prediction3 = model3.predict(x_test)
print("Matriz de confusión:")
cm3_ad= confusion_matrix(y_test,prediction3)
print(cm2_ad)
print('---------------------------')
print('Exactitud:', accuracy_score(y_test, prediction3))
print('---------------------------')
print('Precision Score:', precision_score(y_test, prediction3))
print('---------------------------')
print('Recall Score:', recall_score(y_test, prediction3))
print('---------------------------')
#Reporte de Clasificacion
#print("Reporte de Clasificación:")
#print('---------------------------')
#print(classification_report(y_test, prediction3))

### Modelo 4: ***XGBoost***

In [ ]:
#librerias 
#!pip install xgboost
import xgboost

In [ ]:
cont=0
for i in [df_train_1,df_train_2,df_train_3]:
    print('================================================')
    cont+=1
    if cont==1:
        print('Analisis numero 1')
        print('Modelo con dataset eliminando datos faltantes ya que hay baja correlacion con la variable a predecir y tienen poco peso sobre el total de los datos')
        print('================================================')
    elif cont==2:
        print('Analisis numero 2')
        print('Modelo con dataset reemplazando los valores faltantes por mediana en caso de series numericas y "Desconocido" para categoricas.')
        print('================================================')
    else:
        print('Analisis numero 3')
        print('Modelo con dataset reemplazando los valores faltantes numericos por el promedio y "Desconocido" para categoricas.')
        print('================================================')
    #armo lista de variables categoricas para transformar valor a frecuencias
    categoricas = []
    for j in i.columns:
        if i[j].dtype == 'O':
            categoricas.append(j)
    for cat in categoricas:
        i[cat]=i.groupby(cat)[cat].transform('count') #transformo valor a frecuencia
    i.head()
    # variables explicativas
    x_train_ml = i.loc[:, i.columns != 'Subscription'] 
    # variables de respuesta
    y_train_ml = i[['Subscription']] 
    #separo la muestra en train y test
    x_train, x_test, y_train, y_test = train_test_split(x_train_ml, 
                                          y_train_ml,
                                          test_size=0.25,
                                          random_state=0)
    #separo prueba y validacion
    x_valid, x_test, y_valid, y_test = train_test_split(x_test, 
                                          y_test,
                                          test_size=0.50,
                                          random_state=0)
    #import xgboost
    xgb1 = xgboost.XGBClassifier(eta=0.5,max_depth=10)
    xgb2 = xgboost.XGBClassifier(eta=0.6,max_depth=20)
    xgb3 = xgboost.XGBClassifier(eta=0.7,max_depth=30)
    xgb4 = xgboost.XGBClassifier(eta=0.8,max_depth=40)
    #optimizar parametros
    '''params = {'nthreads':[1], # numero de hilos
              'objective':['binary:logistic'],# tipo de clasificacion entre 0 y 1
              'learning_rate':[0.05,0.1], #tasa de aprendizaje, se entrena con ambos
              'n_estimators':[100,200]} # numero de arboles, boosting basado en arboles, corre para cada valor.
    #importo para hacer validacion cruzada
    #from sklearn.model_selection import GridSearchCV
    #parametros de entrenamiento para que no haya sobreentrenamiento
    #dar valor de corte
    fit_params = {'early_stoping_round':10, # si en 10 rondas no mejora, detiene entrenamiento
                  'eval_metric':'logloss', #funcion de perdida logaritmica   
                  'eval_set':[(x_test, y_test)]}
    clf_xgb = GridSearchCV(xgb, params, 
                       fit_params=fit_params,
                       cv=3, #3 pliegues por todas las combinaciones seteadas en params
                       scoring='accuracy')'''
    xgb1.fit(x_train,y_train)
    xgb2.fit(x_train,y_train)
    xgb3.fit(x_train,y_train)
    xgb4.fit(x_train,y_train)
    #best_xgb = xgb.best_estimator_
    #print('Mejor estimador: %s'%(best_xgb))
    #print('Mejor score: %s'%(xgb.best_score_))
    #predicciones con valid
    y_pred_xgb1 = xgb1.predict(x_valid)
    y_pred_xgb2 = xgb2.predict(x_valid)
    y_pred_xgb3 = xgb3.predict(x_valid)
    y_pred_xgb4 = xgb4.predict(x_valid)
    #armo df con reales y predichos
    #comparar = pd.DataFrame({'real':y_valid,'predicho':y_pred_xgb})
    print('---------------------------')
    print("Matriz de confusión: eta=0.5,max_depth=10")
    print('---------------------------')
    cm_xgb1= confusion_matrix(y_valid,y_pred_xgb1)
    print(cm_xgb1)
    #accuracy
    print('Exactitud: ',accuracy_score(y_valid,y_pred_xgb1))
    #se puede mejorar aumentando la cantidad de arboles pero aumenta el tiempo de procesamiento
    #tambien puede haber sobre entrenamiento con tantos arboles
    print('---------------------------')
    print("Matriz de confusión: eta=0.6,max_depth=20")
    print('---------------------------')
    cm_xgb2= confusion_matrix(y_valid,y_pred_xgb2)
    print(cm_xgb2)
    #accuracy
    print('Exactitud: ',accuracy_score(y_valid,y_pred_xgb2))
    print('---------------------------')
    print("Matriz de confusión: eta=0.7,max_depth=30")
    print('---------------------------')
    cm_xgb3= confusion_matrix(y_valid,y_pred_xgb3)
    print(cm_xgb3)
    #accuracy
    print('Exactitud: ',accuracy_score(y_valid,y_pred_xgb3))
    print('---------------------------')
    print("Matriz de confusión: eta=0.8,max_depth=40")
    print('---------------------------')
    cm_xgb4= confusion_matrix(y_valid,y_pred_xgb4)
    print(cm_xgb4)
    #accuracy
    print('Exactitud: ',accuracy_score(y_valid,y_pred_xgb4))
    
    

A partir de estos resultados observamos que el **XGBoost con los hiperparámetros de eta=0.6 y max_depth = 20**, es el mejor modelo para predecir en este caso:

In [ ]:
del df_test['Total_loan'] # lo sacamos porque daba la mismo info que Deudor

In [ ]:
# predicciones finales para el XGBoost

cont=0
for i in [df_train_2]:
    print('================================================')
    cont+=1
    if cont==1:
        print('Analisis numero 1')
        print('Modelo con dataset eliminando datos faltantes ya que hay baja correlacion con la variable a predecir y tienen poco peso sobre el total de los datos')
        print('================================================')
    elif cont==2:
        print('Analisis numero 2')
        print('Modelo con dataset reemplazando los valores faltantes por mediana en caso de series numericas y "Desconocido" para categoricas.')
        print('================================================')
    else:
        print('Analisis numero 3')
        print('Modelo con dataset reemplazando los valores faltantes numericos por el promedio y "Desconocido" para categoricas.')
        print('================================================')
    #armo lista de variables categoricas para transformar valor a frecuencias
    categoricas = []
    for j in i.columns:
        if i[j].dtype == 'O':
            categoricas.append(j)
    for cat in categoricas:
        i[cat]=i.groupby(cat)[cat].transform('count') #transformo valor a frecuencia
    i.head()
    # variables explicativas
    x_train_ml = i.loc[:, i.columns != 'Subscription'] 
    # variables de respuesta
    y_train_ml = i[['Subscription']] 
    #separo la muestra en train y test
    x_train, x_test, y_train, y_test = train_test_split(x_train_ml, 
                                          y_train_ml,
                                          test_size=0.25,
                                          random_state=0)
    #separo prueba y validacion
    x_valid, x_test, y_valid, y_test = train_test_split(x_test, 
                                          y_test,
                                          test_size=0.50,
                                          random_state=0)
    #import xgboost
    xgb1 = xgboost.XGBClassifier(eta=0.5,max_depth=10)
    xgb2 = xgboost.XGBClassifier(eta=0.6,max_depth=20)
    xgb3 = xgboost.XGBClassifier(eta=0.7,max_depth=30)
    xgb4 = xgboost.XGBClassifier(eta=0.8,max_depth=40)
    #optimizar parametros
    '''params = {'nthreads':[1], # numero de hilos
              'objective':['binary:logistic'],# tipo de clasificacion entre 0 y 1
              'learning_rate':[0.05,0.1], #tasa de aprendizaje, se entrena con ambos
              'n_estimators':[100,200]} # numero de arboles, boosting basado en arboles, corre para cada valor.
    #importo para hacer validacion cruzada
    #from sklearn.model_selection import GridSearchCV
    #parametros de entrenamiento para que no haya sobreentrenamiento
    #dar valor de corte
    fit_params = {'early_stoping_round':10, # si en 10 rondas no mejora, detiene entrenamiento
                  'eval_metric':'logloss', #funcion de perdida logaritmica   
                  'eval_set':[(x_test, y_test)]}
    clf_xgb = GridSearchCV(xgb, params, 
                       fit_params=fit_params,
                       cv=3, #3 pliegues por todas las combinaciones seteadas en params
                       scoring='accuracy')'''
    xgb1.fit(x_train,y_train)
    xgb2.fit(x_train,y_train)
    xgb3.fit(x_train,y_train)
    xgb4.fit(x_train,y_train)
    #best_xgb = xgb.best_estimator_
    #print('Mejor estimador: %s'%(best_xgb))
    #print('Mejor score: %s'%(xgb.best_score_))
    #predicciones con valid
    y_pred_xgb1 = xgb1.predict(x_valid)
    y_pred_xgb2 = xgb2.predict(x_valid)
    y_pred_xgb3 = xgb3.predict(x_valid)
    y_pred_xgb4 = xgb4.predict(x_valid)
    #armo df con reales y predichos
    #comparar = pd.DataFrame({'real':y_valid,'predicho':y_pred_xgb})
    print('---------------------------')
    print("Matriz de confusión: eta=0.5,max_depth=10")
    print('---------------------------')
    cm_xgb1= confusion_matrix(y_valid,y_pred_xgb1)
    print(cm_xgb1)
    #accuracy
    print('Exactitud: ',accuracy_score(y_valid,y_pred_xgb1))
    #se puede mejorar aumentando la cantidad de arboles pero aumenta el tiempo de procesamiento
    #tambien puede haber sobre entrenamiento con tantos arboles
    print('---------------------------')
    print("Matriz de confusión: eta=0.6,max_depth=20")
    print('---------------------------')
    cm_xgb2= confusion_matrix(y_valid,y_pred_xgb2)
    print(cm_xgb2)
    #accuracy
    print('Exactitud: ',accuracy_score(y_valid,y_pred_xgb2))
    print('---------------------------')
    print("Matriz de confusión: eta=0.7,max_depth=30")
    print('---------------------------')
    cm_xgb3= confusion_matrix(y_valid,y_pred_xgb3)
    print(cm_xgb3)
    #accuracy
    print('Exactitud: ',accuracy_score(y_valid,y_pred_xgb3))
    print('---------------------------')
    print("Matriz de confusión: eta=0.8,max_depth=40")
    print('---------------------------')
    cm_xgb4= confusion_matrix(y_valid,y_pred_xgb4)
    print(cm_xgb4)
    #accuracy
    print('Exactitud: ',accuracy_score(y_valid,y_pred_xgb4))
    
    categoricas_2 = []
    for j in df_test.columns:
        if df_test[j].dtype == 'O':
            categoricas_2.append(j)
    for cat in categoricas_2:
        df_test[cat]=df_test.groupby(cat)[cat].transform('count') #transformo valor a frecuencia
    df_test.head()
    
    y_pred_xgb2_final = xgb2.predict(df_test)


In [ ]:
df_test['y_Subscrip_predichos'] = y_pred_xgb2_final

In [ ]:
df_test.head()

In [ ]:
#exportamos los valores predichos finales

df_test.to_csv('Predicciones finales Subscriptions.csv',index=False)